In [1]:
import json
import csv
from google.colab import drive
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Dense
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report, f1_score
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')
import sys
csv.field_size_limit(sys.maxsize)
import random

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
maxLength = 0
text = []
classification = []
temp = []
with open('/content/drive/My Drive/cs3244/fulltrain.csv','r') as f:
  data = csv.reader(f)
  for row in data:
    temp.append((row[0],row[1]))
random.shuffle(temp)
for i in range(500):
  text.append(temp[i][1])
  if len(temp[i][1].split()) > maxLength:
    maxLength = len(temp[i][1].split())
  classification.append(int(temp[i][0])-1)
print(classification)
print(maxLength)

[3, 0, 0, 0, 2, 0, 2, 2, 2, 1, 2, 2, 1, 2, 3, 0, 0, 0, 2, 3, 2, 3, 0, 2, 2, 1, 3, 3, 1, 2, 1, 3, 0, 0, 0, 0, 3, 0, 0, 1, 2, 2, 0, 0, 2, 2, 2, 3, 1, 1, 1, 2, 3, 3, 2, 1, 3, 1, 1, 2, 3, 1, 2, 2, 0, 3, 2, 1, 2, 3, 2, 2, 0, 3, 2, 0, 3, 1, 3, 2, 1, 2, 2, 2, 2, 2, 1, 3, 0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 1, 0, 3, 1, 1, 3, 0, 2, 2, 2, 3, 2, 0, 0, 1, 3, 2, 1, 0, 1, 0, 2, 2, 2, 2, 1, 0, 2, 1, 0, 1, 1, 3, 0, 2, 2, 3, 1, 2, 3, 0, 0, 2, 0, 0, 2, 0, 1, 0, 3, 2, 2, 1, 3, 2, 0, 3, 2, 2, 2, 2, 0, 0, 1, 3, 0, 1, 3, 0, 2, 2, 2, 2, 0, 3, 2, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 1, 0, 1, 2, 2, 3, 2, 3, 1, 2, 2, 3, 2, 3, 0, 2, 3, 1, 3, 3, 2, 1, 3, 1, 3, 2, 1, 0, 0, 3, 2, 2, 1, 2, 1, 2, 0, 0, 2, 2, 2, 1, 0, 3, 2, 2, 3, 0, 2, 3, 3, 2, 2, 1, 2, 2, 3, 2, 0, 2, 2, 2, 1, 0, 2, 2, 2, 3, 0, 1, 2, 2, 1, 2, 0, 2, 1, 1, 2, 1, 2, 0, 2, 2, 2, 0, 2, 0, 0, 1, 1, 2, 3, 0, 1, 2, 0, 2, 3, 0, 2, 3, 1, 2, 0, 2, 2, 0, 3, 0, 2, 0, 1, 2, 3, 3, 0, 0, 2, 2, 0, 2, 3, 2, 0, 1, 3, 0, 3, 0, 3, 0, 0, 2, 0, 0, 3, 1, 2, 1, 3, 0, 2, 2, 1, 2, 0, 0, 

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
X = tokenizer.texts_to_sequences(text)
X = pad_sequences(X, padding = "post", truncating = "post", maxlen = maxLength)
y = [to_categorical(i,num_classes = 4) for i in classification]
#y = classification

In [4]:
embed = {}
f = open('drive/MyDrive/glove.6B.100d.txt','r')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype = "float32")
	embed [word] = coefs
f.close()

In [5]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embed.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [6]:
print(X.shape)
print(np.array(y).shape)

keywordModel = Sequential()
keywordModel.add(Embedding(len(word_index) + 1, 100,input_length=maxLength, weights = [embedding_matrix]))
keywordModel.add(Bidirectional(LSTM(100)))
keywordModel.add(Dense(100, activation = "relu"))
keywordModel.add(Dense(4, activation = "softmax"))
keywordModel.compile(loss="categorical_crossentropy", optimizer = "adam")
keywordModel.fit(X, np.array(y), batch_size = 32, epochs = 10, validation_split = 0.1)


(500, 8384)
(500, 4)
Epoch 1/10
15/15 [==============================] - 310s 20s/step - loss: 1.3082 - val_loss: 1.2402
Epoch 2/10
15/15 [==============================] - 304s 20s/step - loss: 1.0670 - val_loss: 1.1149
Epoch 3/10
15/15 [==============================] - 299s 20s/step - loss: 0.8385 - val_loss: 0.8821
Epoch 4/10
15/15 [==============================] - 303s 20s/step - loss: 0.6595 - val_loss: 0.9729
Epoch 5/10
15/15 [==============================] - 307s 21s/step - loss: 0.5476 - val_loss: 0.8046
Epoch 6/10
15/15 [==============================] - 305s 20s/step - loss: 0.3735 - val_loss: 0.6259
Epoch 7/10
15/15 [==============================] - 301s 20s/step - loss: 0.2144 - val_loss: 0.7196
Epoch 8/10
15/15 [==============================] - 306s 20s/step - loss: 0.1343 - val_loss: 0.8344
Epoch 9/10
15/15 [==============================] - 298s 20s/step - loss: 0.0881 - val_loss: 0.6786
Epoch 10/10
15/15 [==============================] - 297s 20s/step - loss: 0.05

In [7]:
textTest = []
classificationTest = []
with open('/content/drive/My Drive/cs3244/balancedtest.csv','r') as f:
  data = csv.reader(f)
  for row in data:
    temp.append((row[0],row[1]))
random.shuffle(temp)
for i in range(500):
  textTest.append(temp[i][1])
  classificationTest.append(int(temp[i][0])-1)
XTest = tokenizer.texts_to_sequences(textTest)
XTest = pad_sequences(XTest, padding = "post", truncating = "post", maxlen = maxLength)
yTest = [to_categorical(i,num_classes = 4) for i in classificationTest]

In [8]:
print(XTest.shape)
a = keywordModel.predict(XTest)
a = np.argmax(a, axis = -1)
flattened_actual = (np.argmax(np.array(yTest), axis = -1)).flatten()
flattened_output = a.flatten()
print(classification_report(flattened_actual, flattened_output))
print(f1_score(y_pred=flattened_actual, y_true=flattened_output,average = "macro"))

(500, 8384)
              precision    recall  f1-score   support

           0       0.79      0.77      0.78       135
           1       0.65      0.80      0.72        89
           2       0.82      0.74      0.78       171
           3       0.72      0.72      0.72       105

    accuracy                           0.75       500
   macro avg       0.75      0.76      0.75       500
weighted avg       0.76      0.75      0.76       500

0.7488480183985803
